# Multiprocessing vs Multithreading vs Asyncio in Python
Python provides several ways to perform concurrent or parallel processing. The three most commonly used approaches are multiprocessing, multithreading, 
and asyncio. In this notebook, we'll compare these three approaches and discuss their advantages and disadvantages.

## 1. Introduction to multiprocessing, multithreading, and asyncio in Python
### Multiprocessing
Multiprocessing is a technique where a program uses multiple processors to run several tasks in parallel. In Python, the multiprocessing module provides support for multiprocessing. Each process has its own memory space and runs independently of other processes.

### Multithreading
Multithreading is a technique where a program uses multiple threads to run several tasks concurrently. In Python, the threading module provides support for multithreading. Threads share the same memory space and run within the same process.

### Asyncio
Asyncio is a technique where a program uses cooperative multitasking to run several tasks concurrently. In Python, the asyncio module provides support for asyncio. Coroutines are used to represent asynchronous tasks, and the asyncio event loop schedules and runs these tasks.

## 2. Differences between multiprocessing, multithreading, and asyncio
### Execution model
Multiprocessing executes tasks in parallel, with each task running in a separate process.

Multithreading executes tasks concurrently, with multiple threads running within the same process.

Asyncio executes tasks cooperatively, with coroutines running within the same thread and the asyncio event loop scheduling and running the coroutines.

### Overhead
Multiprocessing has the highest overhead, as it requires creating new processes and transferring data between them.

Multithreading has less overhead than multiprocessing, as threads share the same memory space.

Asyncio has the lowest overhead, as it uses cooperative multitasking within the same thread.

Parallelism
Multiprocessing provides true parallelism, as tasks run in separate processes and can utilize multiple CPUs.

Multithreading provides concurrency, but not true parallelism, as threads run within the same process and share the same resources.

Asyncio provides concurrency, but not true parallelism, as coroutines run within the same thread and share the same resources.

### Complexity
Multiprocessing is more complex than multithreading and asyncio, as it involves managing multiple processes and communication between them.

Multithreading is less complex than multiprocessing, but still requires careful management of shared resources.

Asyncio is the simplest approach, as it only requires managing coroutines within the same thread.



## Examples and code snippets to illustrate the differences between the three approaches
### Multiprocessing example
Here we have a function that runs the recursive function to find a specific Fibonacci number, a certain number of times according to the user's choice

#### without multiprocessing: 

In [2]:
import time

times = 2

def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)

def calculate(n, amount=1):
    return (f"calculate {amount} times", [fib(n) for _ in range(amount)])
  

time_start = time.time()
print(calculate(31, times))
time_end = time.time()
print(f'Time elapsed: {time_end - time_start} seconds')

('calculate 2 times', [1346269, 1346269])
Time elapsed: 0.516420841217041 seconds


#### with multiprocessing:

In [3]:
import multiprocessing as mp

def calculate(n, amount=1):
    numbers = [n] * amount
    pool = mp.Pool()
    results = pool.map(fib, numbers)
    return (f"calculate {amount} times", results)

time_start = time.time()
results = calculate(31, times)
time_end = time.time()
print(results)
print(f'Time elapsed: {time_end - time_start} seconds')

('calculate 2 times', [1346269, 1346269])
Time elapsed: 0.39865970611572266 seconds


#### and with Multithreading:

In [6]:
import threading

def calculate(n, amount=1):
    numbers = [n] * amount
    threads = []
    results = []
    for number in numbers:
        thread = threading.Thread(target=lambda: results.append(fib(number)))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    return (f"calculate {amount} times", results)

time_start = time.time()
results = calculate(31, times)
time_end = time.time()
print(results)
print(f'Time elapsed: {time_end - time_start} seconds')

('calculate 2 times', [1346269, 1346269])
Time elapsed: 0.5165376663208008 seconds


### and now the Multithreading va asyncio axample
#### We have a function here that simulates preparing breakfast with the components of: omelette, toast, and salad. and the preparation time of each of them.

##### without Multithreading and asyncio:

In [7]:
def omelet():
    time.sleep(3)
    print('1 omelet is ready')

def toast():
    time.sleep(2)
    print('1 toast is ready')

def salad():
    time.sleep(5)
    print('1 salad is ready')

def breakfast(persons=1):
    for _ in range(persons):
        omelet()
        salad()
        toast()
        toast()
    print('Breakfast is ready') 
       
def main():
    time_start = time.time()
    breakfast()
    time_end = time.time()
    print(f'Time elapsed: {time_end - time_start} seconds')
    
if __name__ == '__main__':
    main()

1 omelet is ready
1 salad is ready
1 toast is ready
1 toast is ready
Breakfast is ready
Time elapsed: 12.00125002861023 seconds


#### with Multithreading:

In [8]:
import threading

def omelet():
    time.sleep(3)
    print('1 omelet is ready')

def toast():
    time.sleep(2)
    print('1 toast is ready')

def salad():
    time.sleep(5)
    print('1 salad is ready')

def breakfast(persons=1):
    threads = []
    for _ in range(persons):
        t1 = threading.Thread(target=omelet)
        t2 = threading.Thread(target=salad)
        t3 = threading.Thread(target=toast)
        t4 = threading.Thread(target=toast)
        threads.extend((t1, t4, t3, t2))
        t2.start()
        t3.start()
        t4.start()
        t1.start()
    for t in threads:
        t.join()
    print('Breakfast is ready')

def main():
    time_start = time.time()
    breakfast()
    time_end = time.time()
    print(f'Time elapsed: {time_end - time_start} seconds')

if __name__ == '__main__':
    main()

1 toast is ready
1 toast is ready
1 omelet is ready
1 salad is ready
Breakfast is ready
Time elapsed: 5.002636432647705 seconds


#### and now with asyncio:
in order to use asyncio with Jupiter,
We must use "%%file" to write the code to a file.

In [10]:
# %%file demo_asyncio.py
import asyncio as aio
import time

async def omelet():
    await aio.sleep(3)
    print('1 omelet is ready')

async def toast():
    await aio.sleep(2)
    print('1 toast is ready')

async def salad():
    await aio.sleep(5)
    print('1 salad is ready')

async def breakfast(persons=1):
    await aio.gather(
        *[toast() for _ in range(persons * 2)],
        *[salad() for _ in range(persons)],
        *[omelet() for _ in range(persons)],
    )
    print('Breakfast is ready')

def main():
    time_start = time.time()
    aio.run(breakfast(20))
    time_end = time.time()
    print(f'Time elapsed: {time_end - time_start} seconds')

if __name__ == "__main__":
    main()


Overwriting demo_asyncio.py


and Then run with the command line:

In [12]:
!python3 demo_asyncio.py

1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 toast is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is ready
1 omelet is re

### so to the conclusion:
# 3. Advantages and disadvantages of multiprocessing, multithreading, and asyncio
## Multiprocessing
### Advantages:

* Provides true parallelism, which can result in significant performance gains for CPU-bound tasks.
* Each process has its own memory space, which can prevent issues with shared resources.
### Disadvantages:

* High overhead due to process creation and data transfer between processes.
* Complex management of multiple processes.
## Multithreading
### Advantages:

* Low overhead, as threads share the same memory space.
* Can improve performance for I/O-bound tasks.
### Disadvantages:

* Difficult to manage shared resources, such as variables and data structures.
* Limited scalability due to the global interpreter lock (GIL), which prevents multiple threads from executing Python bytecode simultaneously.
## Asyncio
### Advantages:

* Low overhead, as coroutines run within the same thread.
* Can handle large numbers of I/O-bound tasks with ease.
### Disadvantages:

* Limited to I/O-bound tasks, as CPU-bound tasks can block the event loop.
* Not suitable for tasks that require true parallelism.

## 4. When to use multiprocessing, multithreading, and asyncio
### Use multiprocessing when:
* You need to perform CPU-bound tasks that can benefit from true parallelism.
* You need to prevent issues with shared resources by running tasks in separate processes.
* The overhead of creating and managing multiple processes is acceptable.

### Use multithreading when:

* You need to perform I/O-bound tasks that can benefit from concurrency.
* You can manage shared resources carefully to prevent issues such as race conditions.
* The limitations of the GIL are not a significant concern.

### Use asyncio when:

* You need to handle large numbers of I/O-bound tasks with minimal overhead.
* You can structure your code using coroutines and the asyncio event loop.
* You do not need true parallelism or do not want to deal with the overhead and complexity of multiprocessing.